[1. dennybritz](https://github.com/dennybritz/reinforcement-learning/blob/master/PolicyGradient/Continuous%20MountainCar%20Actor%20Critic%20Solution.ipynb) <br>
[2. original](https://gist.github.com/Ashioto/10ec680395db48ddac1ad848f5f7382c#file-actorcritic-py)

In [1]:
%reload_ext autoreload
%autoreload 2
import torch
import random
import numpy as np
from EXITrl.approx_v_base import ApproxVBase
from EXITrl.approx_policy_base import ApproxPolicyBase
from EXITrl.base import Base
from EXITrl.helpers import print_weight_size
import gym

### A2C [Continuous]

In [2]:
import sklearn.pipeline
import sklearn.preprocessing
from sklearn.kernel_approximation import RBFSampler
# Feature Preprocessing: Normalize to zero mean and unit variance
# We use a few samples from the observation space to do this
env = gym.make('MountainCarContinuous-v0')
observation_examples = np.array([env.observation_space.sample() for x in range(10000)])
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(observation_examples)

# Used to converte a state to a featurizes represenation.
# We use RBF kernels with different variances to cover different parts of the space
featurizer = sklearn.pipeline.FeatureUnion([
        ("rbf1", RBFSampler(gamma=5.0, n_components=100)),
        ("rbf2", RBFSampler(gamma=2.0, n_components=100)),
        ("rbf3", RBFSampler(gamma=1.0, n_components=100)),
        ("rbf4", RBFSampler(gamma=0.5, n_components=100))
        ])
featurizer.fit(scaler.transform(observation_examples))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/Users/epinyoanun/miniconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


FeatureUnion(n_jobs=1,
       transformer_list=[('rbf1', RBFSampler(gamma=5.0, n_components=100, random_state=None)), ('rbf2', RBFSampler(gamma=2.0, n_components=100, random_state=None)), ('rbf3', RBFSampler(gamma=1.0, n_components=100, random_state=None)), ('rbf4', RBFSampler(gamma=0.5, n_components=100, random_state=None))],
       transformer_weights=None)

In [3]:
def featurize_state(state):
    """
    Returns the featurized representation for a state.
    """
    scaled = scaler.transform([state])
    featurized = featurizer.transform(scaled)
    return featurized[0]
state = env.reset()
featurize_state(state).shape

(400,)

In [6]:
from torch import nn
class GaussianPolicyModel(nn.Module):
    """TODO add to device()"""
    def __init__(self, num_input=2, num_output=1):
        super().__init__()
        # 1. mu
        self.model_mu = torch.nn.Linear(num_input, num_output)
        # 2. sigma
        self.model_sigma = torch.nn.Sequential(
            torch.nn.Linear(num_input, num_output),
            torch.nn.Softplus() # make it positive, be careful threshold=20???
        )
        self.sigma = 2

    def forward(self, x):
        mu = self.model_mu(x)
        sigma = self.model_sigma(x)
        return mu, self.sigma
policy = GaussianPolicyModel(2)
optimizer_sigma = torch.optim.Adam(policy.parameters(), lr=0.5)

In [5]:
class REINFORCE_continuous(ApproxVBase, ApproxPolicyBase):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        num_feature = 400
        model_q = torch.nn.Linear(num_feature, 1)
        ApproxVBase.initialize(self, learning_rate_name="beta", model=model_q)
        self.model_g = GaussianPolicyModel(num_feature, self.num_action)
        ApproxPolicyBase.initialize(self, 
                                    learning_rate_name="alpha", 
                                    model=self.model_g)
        
    def _loop(self, episode) -> int:
        done = False
        total_reward, reward = 0, 0
        state = self.env.reset()
        state = featurize_state(state)
        # decay sigma to have less exploration
        self.model_g.sigma *= 0.9
        self.additional_log['sigma'] = self.model_g.sigma
        for i in range(1000):
            action, log_prob, entropy = self.policy(state)
#             print('\raction:',action,'***********' , end="")
            _state, reward, done, _ = self.env.step(action)
            _state = featurize_state(_state)
#             self.env.render()
            
            if done: 
                td_target = reward
            else: 
                td_target = reward + self.gamma * self.get_v(_state)
            estimate_v = self.get_v(state)
            td_error = td_target - estimate_v
            
            self.update_v(td_target, self.get_v(state))
            loss = (-log_prob) * td_error # using the td error as our advantage estimate
            self.update_policy(loss)

            total_reward += reward
            state = _state
            if done: return total_reward

try: env.close()
except: pass
env = gym.make('MountainCarContinuous-v0')
s = REINFORCE_continuous(env, 
               num_episodes=50,
               policy="gaussian_policy",
               alpha=0.0001, 
               beta=0.001,
               gamma=.99)
s.train(True)

/Users/epinyoanun/miniconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Episode 10	Average Score: 25.02 	other{'sigma': 0.6973568802000003}
Episode 20	Average Score: 66.81 	other{'sigma': 0.24315330918113873}
Episode 30	Average Score: 89.11 	other{'sigma': 0.08478231655043249}
Episode 40	Average Score: 20.82 	other{'sigma': 0.029561765882869215}
Episode 50	Average Score: 58.81 	other{'sigma': 0.01030755041464024}}


In [11]:
env.close()